# Integración numérica de EDO acopladas

Esta notebook es una excusa para aprender integración numérica de ecuaciones en Python. El sistema que se considerará es dos masas acopladas a tres resortes.

<img src="./resorte_doble.gif">

Las ecuaciones de movimiento son:

$$ m_1 \frac{d^2 x_1}{d t^2} = -k_1 x_1 + k_c (x_2 - x_1) $$

$$ m_2 \frac{d^2 x_2}{d t^2} = -k_2 x_2 - k_c (x_2 - x_1) $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def OAA(X, t, k1, k2, kc, m1, m2):
    """Osciladores Armónimos Acoplados"""
    x1, x2 , v1 , v2 = X
    x1p = v1
    v1p = (-k1*x1 + kc*(x2-x1))/m1
    x2p = v2
    v2p = (-k2*x2 - kc*(x2-x1))/m2
    return x1p, x2p, v1p ,v2p

In [ ]:
#Condiciones iniciales : Desplazamientos
x1o, x2o = 1.0 , 0.0
#Condiciones iniciales : Velocidades
v1o, v2o = 0.0 , 0.0

In [ ]:
# Parámetros
m1 , m2 = 1.0 , 1.0
k1 , k2 = 10.0 ,10.0
kc = 0.5

In [ ]:
tmax, n = 160, 10000 # tmax: tiempo total. n: puntos de discretización del intervalo de tiempo 
t = np.linspace(0, tmax, n)

In [ ]:
t.size

Integraremos las ecuaciones para la grilla de tiempo. Usaremos el método `odeint` de scipy.

In [ ]:
from scipy.integrate import odeint

In [ ]:
odeint?

Los parámetros son: la función derivada de `y`, las condiciones iniciales `y0`, y el array de tiempos `t`, y los argumentos `args` si hicieran falta pasarle a la función (en este caso las constantes y las masas).

Como resultado devuelve el array `y` resuleto, con `y0` en la primera fila.

Noten que por atrás usa una librería de FORTRAN (odepack), por lo que corre código compilado! Numpy también tiene muchas funciones en FORTRAN.

In [ ]:
f = odeint(OAA, (x1o, x2o, v1o, v2o), t, args=(k1, k2, kc, m1, m2))
x1,x2,v1,v2 = f.T

In [ ]:
plt.plot(t, x1, "g-", linewidth=2, label="x1" )
plt.plot(t, x2, "r-", linewidth=2, label="x2" )
plt.xlabel("$tiempo$")
plt.ylabel("$desplazameinto$")
xmin, xmax = -0.5, tmax
ymin, ymax = -2.0, 2.0
plt.axis([xmin,xmax,ymin,ymax])
plt.legend(loc="upper left") # 
#plt.savefig("myplot.pdf")

In [ ]:
fig, ax = plt.subplots(2, figsize=(16,4), sharex=True, sharey=True)
ax[0].plot(t, x1, 'b-', lw=2, label="x1")
ax[1].plot(t, x2, 'r-', lw=2, label="x2")
ax[0].set_xlim(0,tmax)
ax[0].set_ylim(-1.5, 1.5)
ax[1].set_xlabel('tiempo / s', fontsize=20)
ax[0].set_ylabel('$\Delta x_1$', fontsize=20)
ax[1].set_ylabel('$\Delta x_2$', fontsize=20)

In [ ]:
plt.figure(figsize=(4,4))
plt.plot(x1, x2)
plt.axis('equal')
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

También podemos hacer un histograma de cuántas veces "visitó" cada configuración:

In [ ]:
hist, x1edge, x2edge = np.histogram2d(x1, x2, bins=100)

In [ ]:
hist.shape

In [ ]:
hist.max(), hist.min()

In [ ]:
x1edge.min(), x1edge.max(), x2edge.min(), x2edge.max()

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(hist, vmax=hist.max()/6., extent=[x1edge.min(), x1edge.max(), x2edge.min(), x2edge.max()])

Y también podemos graficarlo en 3D:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.gca(projection='3d')

# Arrays de posición de cada una de las barras.
# Como np.meshgrid da arrays en (ny, nx) y hay que "aplanarlos" usando indexado de Fortran ("F") (column-mayor order)
xpos, ypos = np.meshgrid(x1edge[:-1] + 0.1, x2edge[:-1] + 0.1)
#xpos = xpos.flatten('F')
#ypos = ypos.flatten('F')
#zpos = np.zeros_like(xpos)

# Construct arrays with the dimensions for the 16 bars.
dx = 0.5 * np.ones_like(zpos)
dy = dx.copy()
dz = hist.flatten()

#ax.bar3d(xpos, ypos, zpos, dx, dy, dz, color='b', zsort='average')
ax.plot_surface(xpos, ypos, hist, cmap=plt.get_cmap("viridis"), antialiased=False)

Queremos saber qué frecuencias están presentes en los desplazamientos. Para eso debemos tomar la transformada de Fourier de la señal. Esto se hace fácilmente usando la FFT (fast Fourier transform) de numpy:

In [ ]:
freq = np.fft.rfftfreq?
# se utiliza para generar el array de frecuencias

In [ ]:
freq = np.fft.rfft?
#se utiliza para calcular la transformada de la señal. Distinta sintaxis que la anterior!

In [ ]:
padding = 10

In [ ]:
freq = np.fft.rfftfreq(padding * t.size, t[1]-t[0])

In [ ]:
x1_freq = np.fft.rfft(x1-x1[0], padding * t.size) #el resultado es complejo

In [ ]:
plt.plot(freq, abs(x1_freq))
plt.xlim(0.4, 0.6)
plt.ylim(0,3000)

Resolviendo analíticamente sabemos que presenta dos modos normales. Por simplicidad consideramos el caso $k_1=k_2=k$ y $m_1 = m_2 = m$. Los modos tienen frecuencias:

$$\omega_1 = \sqrt{\frac{k}{m}}$$

$$\omega_2 = \sqrt{\frac{k+2k_c}{m}}$$

Podemos graficar las frecuencias analíticas sobre la transformada de Fourier anterior para corroborar:

In [ ]:
omega1 = np.sqrt(k1/m1)
omega2 = np.sqrt((k1+2*kc)/m1)

In [ ]:
print(omega1, omega2)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(freq, abs(x1_freq))
plt.xlim(0.4, 0.6)
plt.ylim(0,3000)
plt.axvline(omega1/(2*np.pi), c='r', linestyle='--')
plt.axvline(omega2/(2*np.pi), c='r', linestyle='--')
plt.xlabel('frecuencia')
plt.ylabel('intensidad')